# 6-3. 베이즈정리로 텍스트 분류하기

### 텍스트 분류


자주 사용되는 방법으로 베이즈 정리를 이용한 텍스트 분류 방법인 "베이지안 필터"가 있다. 학습을 많이 시킬수록 필터의 분류 능력이 오른다
- 스팸메일 구분하거나 스팸글 구분에 많이 사용됨

* 베이지안 필터는 지도학습
* 머신러닝 이전에는 키워드를 지정해서 스팸메일을 구분했으나, 베이지안 필터를 사용하면 데이터를 학습시켜 자동으로 스팸 메일을 분류할 수 있다(키워드 지정x)

# 베이즈 정리



* 베이즈 정리(Bayes' theorem)은 '조건부 확률'과 관련된 이론(베이지안 필터의 기반)

### 조건부 확률
<img src = "http://cfile3.uf.tistory.com/image/99BBBA335A1CE3AB202A74">

* 사건 B 가 발생한 경우 A 의 확률

* 베이즈 정리는 P(A|B) 의 추정이 P(A∩B) 와 P(B) 에 기반을 두어야한다는 정리

### 베이즈 정리
<img src = "http://pds25.egloos.com/pds/201412/16/00/a0234900_548f230d5e37e.png">

## 나이브 베이즈 분류

 * 문서를 여러 범주 중 하나로 판단하는 문제에 쓰인다

-베이즈 정리는 A라는 사건이 B에 속하는지를 판단할 때 사용
-> 텍스트 분류를 예로 들면 A가 받은메일, B가 스팸 판정결과
   
   * 받은메일에서 스팸일 확률
   
     p(스팸|메일) = { p(스팸)* p(메일|스팸) } / p(메일)
              
              
나이브 베이즈 분류

Naive Bayes 라는 용어는 그 확률모델이 베이즈 정리 (Bayes' Theorem)를 사용하여 유도될 수 있고, 실제로는 생길 수 없는 강한 독립 가정 (strong independences assumption) 를 포함한다는 사실로부터, (의도적으로) naive (순진한, 고지식한) 라는 말을 사용하였다

* 텍스트 내부에서의 단어 출현 비율을 조사,이를 기반으로 해당 텍스트를 어떤 카테고리로 분류하는 것이 적합한지 알아본다.

여러개의 카테고리 중에 어떤 카테고리에 속할 확률이 가장 큰지를 나타내는 정보
(단어가 문서의 어떤 위치에 있다는 정보 등은 고려x. 가방 안에 단어들을 그냥 넣어서 사용하는 것과 같다 BoW)


##### 예시

문서 이진분류 문제를 예

문제 - 문서 d가 주어졌을 때 범주 c1 혹은 c2로 분류하는 것


\begin{align*}
P({ c }_{ 1 }|d)&=\frac { P({ c }_{ 1 },d) }{ P(d) } =\frac { \frac { P({ c }_{ 1 },d) }{ P({ c }_{ 1 }) } \cdot P({ c }_{ 1 }) }{ P(d) } =\frac { P(d|{ c }_{ 1 }) { P({ c }_{ 1 }) }}{ P(d) } \\ P({ c }_{ 2 }|d)&=\frac { P(d|{ c }_{ 2 }){ P({ c }_{ 2 }) } }{ P(d) }
\end{align*}

베이즈 모델은 P(c1|d)/P(d)와 P(c2|d)/P(d)를 비교해 큰 쪽으로 범주를 할당(여기에서 P(d)는 겹치므로 계산을 아예 생략할 수 있습니다)

이번엔 문서 d가 단어 w1, w2로 구성돼 있다고 치자. 
\begin{align*}
P({ c }_{ i }|d)&=P({ c }_{ i }|{ w }_{ 1 },{ w }_{ 2 })\\ &\propto P({ w }_{ 1 },{ w }_{ 2 }|{ c }_{ i }){ P({ c }_{ i }) } \\ &\propto P({ w }_{ 1 },{ w }_{ 2 }|{ c }_{ i })
\end{align*}

나이브 베이즈 분류기는 각 단어가 독립(independent)임을 가정합니다. 모델 이름에 나이브라는 말이 붙은 이유이기도 합니다. 이에 따라 식을 다시 쓸 수 있습니다.

\begin{align*}
P({ w }_{ 1 },{ w }_{ 2 })=P({ w }_{ 1 })\cdot P({ w }_{ 2 })\\ P({ w }_{ 1 },{ w }_{ 2 }|{ c }_{ i })=P({ w }_{ 1 }|{ c }_{ i })\cdot P({ w }_{ 2 }|{ c }_{ i })
\end{align*}

ppt 20 이미지
-> 확률 기반 분류기

In [29]:
from konlpy.tag import Twitter
twitter = Twitter()

train = [("파격 세일 - 오늘까지만 30% 할인", "광고"), 
("쿠폰 선물 & 무료 배송", "광고"),
("현데계 백화점 세일", "광고"),
("봄과 함께 찾아온 따뜻한 신제품 소식", "광고"),
("인기 제품 기간 한정 세일", "광고"),
("오늘 일정 확인", "중요"),
("프로젝트 진행 상황 보고","중요"),
("계약 잘 부탁드립니다","중요"),
("회의 일정이 등록되었습니다.","중요"),
("오늘 일정이 없습니다.","중요")]

In [30]:
def tokenize(doc):
    # norm, stem은 optional
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
train_docs = [(tokenize(row[0]), row[1]) for row in train]
train_docs


[(['파격/Noun',
   '세일/Noun',
   '-/Punctuation',
   '오늘/Noun',
   '까지만/Josa',
   '30/Number',
   '%/Punctuation',
   '할인/Noun'],
  '광고'),
 (['쿠폰/Noun', '선물/Noun', '&/Punctuation', '무료/Noun', '배송/Noun'], '광고'),
 (['현데/Noun', '계/Suffix', '백화점/Noun', '세일/Noun'], '광고'),
 (['봄/Noun',
   '과/Josa',
   '함께/Adverb',
   '찾아오다/Verb',
   '따뜻하다/Adjective',
   '신제품/Noun',
   '소식/Noun'],
  '광고'),
 (['인기/Noun', '제품/Noun', '기간/Noun', '한정/Noun', '세일/Noun'], '광고'),
 (['오늘/Noun', '일정/Noun', '확인/Noun'], '중요'),
 (['프로젝트/Noun', '진행/Noun', '상황/Noun', '보고/Noun'], '중요'),
 (['계약/Noun', '자다/Verb', '부탁드리다/Adjective'], '중요'),
 (['회의/Noun', '일정/Noun', '이/Josa', '등록/Noun', '되어다/Verb', './Punctuation'],
  '중요'),
 (['오늘/Noun', '일정/Noun', '이/Josa', '없다/Adjective', './Punctuation'], '중요')]

In [31]:
#하나의 리스트
tokens = [t for d in train_docs for t in d[0]]
tokens

['파격/Noun',
 '세일/Noun',
 '-/Punctuation',
 '오늘/Noun',
 '까지만/Josa',
 '30/Number',
 '%/Punctuation',
 '할인/Noun',
 '쿠폰/Noun',
 '선물/Noun',
 '&/Punctuation',
 '무료/Noun',
 '배송/Noun',
 '현데/Noun',
 '계/Suffix',
 '백화점/Noun',
 '세일/Noun',
 '봄/Noun',
 '과/Josa',
 '함께/Adverb',
 '찾아오다/Verb',
 '따뜻하다/Adjective',
 '신제품/Noun',
 '소식/Noun',
 '인기/Noun',
 '제품/Noun',
 '기간/Noun',
 '한정/Noun',
 '세일/Noun',
 '오늘/Noun',
 '일정/Noun',
 '확인/Noun',
 '프로젝트/Noun',
 '진행/Noun',
 '상황/Noun',
 '보고/Noun',
 '계약/Noun',
 '자다/Verb',
 '부탁드리다/Adjective',
 '회의/Noun',
 '일정/Noun',
 '이/Josa',
 '등록/Noun',
 '되어다/Verb',
 './Punctuation',
 '오늘/Noun',
 '일정/Noun',
 '이/Josa',
 '없다/Adjective',
 './Punctuation']

In [32]:
#각 docs안에 train
def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in tokens}

train_xy = [(term_exists(d), c) for d, c in train_docs]
train_xy

[({'exists(%/Punctuation)': True,
   'exists(&/Punctuation)': False,
   'exists(-/Punctuation)': True,
   'exists(./Punctuation)': False,
   'exists(30/Number)': True,
   'exists(계/Suffix)': False,
   'exists(계약/Noun)': False,
   'exists(과/Josa)': False,
   'exists(기간/Noun)': False,
   'exists(까지만/Josa)': True,
   'exists(되어다/Verb)': False,
   'exists(등록/Noun)': False,
   'exists(따뜻하다/Adjective)': False,
   'exists(무료/Noun)': False,
   'exists(배송/Noun)': False,
   'exists(백화점/Noun)': False,
   'exists(보고/Noun)': False,
   'exists(봄/Noun)': False,
   'exists(부탁드리다/Adjective)': False,
   'exists(상황/Noun)': False,
   'exists(선물/Noun)': False,
   'exists(세일/Noun)': True,
   'exists(소식/Noun)': False,
   'exists(신제품/Noun)': False,
   'exists(없다/Adjective)': False,
   'exists(오늘/Noun)': True,
   'exists(이/Josa)': False,
   'exists(인기/Noun)': False,
   'exists(일정/Noun)': False,
   'exists(자다/Verb)': False,
   'exists(제품/Noun)': False,
   'exists(진행/Noun)': False,
   'exists(찾아오다/Verb)': False,

In [34]:
import nltk
classifier = nltk.NaiveBayesClassifier.train(train_xy)

classifier.show_most_informative_features(10)

Most Informative Features
         exists(세일/Noun) = False              중요 : 광고     =      2.2 : 1.0
         exists(일정/Noun) = False              광고 : 중요     =      2.2 : 1.0
         exists(오늘/Noun) = True               중요 : 광고     =      1.7 : 1.0
          exists(이/Josa) = False              광고 : 중요     =      1.6 : 1.0
   exists(./Punctuation) = False              광고 : 중요     =      1.6 : 1.0
         exists(오늘/Noun) = False              광고 : 중요     =      1.3 : 1.0
         exists(파격/Noun) = False              중요 : 광고     =      1.2 : 1.0
         exists(배송/Noun) = False              중요 : 광고     =      1.2 : 1.0
        exists(백화점/Noun) = False              중요 : 광고     =      1.2 : 1.0
         exists(인기/Noun) = False              중요 : 광고     =      1.2 : 1.0


In [59]:
test_sentence = [("계약 관련 회의 일정")]
test_docs = tokenize(test_sentence[0])
test_docs

['계약/Noun', '관련/Noun', '회의/Noun', '일정/Noun']

In [60]:
test_sent_feature = {word: (word in tokens) for word in test_docs}
test_sent_feature

{'계약/Noun': True, '관련/Noun': False, '일정/Noun': True, '회의/Noun': True}

In [62]:
classifier.classify(test_sent_feature)

'중요'

## 베이지안 필터 사용해보기

 나이브 베이즈 분류기(Naive Bayes Classifier)를 통해 감성분석

## Naver sentiment movie corpus v1.0


 - Maas et al., 2011 데이터셋 참고
 - 데이터 출처: 네이버
 - 영화 당 100개의 140자평(이하 '리뷰')을 초과하지 않음

 - 크기: 19MB
 - URL: http://github.com/e9t/nsmc/


순서
* KoNLPy 로 데이터 전처리
* NLTK로 데이터 탐색
* BOW로 문서 표현하고 classify
* Gensim으로 doc2vec으로 문서 표현하고 classify

<img src ="https://www.lucypark.kr/docs/2015-pyconkr/ratingdist.png">


 - 총 20만 개 리뷰 (수집된 64만개 중 샘플링)
 - ratings_train.txt: 15만, ratings_test.txt: 5만
 - 긍정/부정 리뷰의 비율을 동일하게 샘플링 (i.e., random guess yields 50% accuracy)
 - 중립 리뷰는 포함하지 않음

## 1. Data preprocessing (feat. KoNLPy)

In [3]:
# 데이터 읽기
import jpype
from konlpy.tag import Twitter


def read_data(filename):
    with open(filename, 'rt', encoding='UTF8') as f:
        data = [line.split('\t') for line in f.read().splitlines()] 
        data = data[1:]   # header 제외
    return data
train_data = read_data('./nsmc-master/ratings_train.txt')
test_data = read_data('./nsmc-master/ratings_test.txt')


In [4]:
# row, column의 수가 제대로 읽혔는지 확인
print(len(train_data))    
print(len(test_data))      

150000
50000


In [5]:
# 형태소로 tokenizing, pos 태깅

from konlpy.tag import Twitter
pos_tagger = Twitter()
def tokenize(doc):
    # norm, stem은 optional
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data]


# 잘 들어갔는지 확인
from pprint import pprint
pprint(train_docs[0])

# => [(['아/Exclamation',
#   '더빙/Noun',
#   '../Punctuation',
#   '진짜/Noun',
#   '짜증/Noun',
#   '나다/Verb',
#   '목소리/Noun'],
#  '0')]

(['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증/Noun',
  '나다/Verb',
  '목소리/Noun'],
 '0')


## 2. Data exploration (feat. NLTK)

In [7]:
#Training data의 token 모으기

tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))
print(tokens[:10])

2194536
['아/Exclamation', '더빙/Noun', '../Punctuation', '진짜/Noun', '짜증/Noun', '나다/Verb', '목소리/Noun', '흠/Noun', '.../Punctuation', '포스터/Noun']


In [8]:
#데이터를 nltk.Text로 묶고
import nltk
text = nltk.Text(tokens)  #nltk.Text()는 문서 하나를 편리하게 탐색할 수 있는 다양한 기능을 제공
print(text)

<Text: 아/Exclamation 더빙/Noun ../Punctuation 진짜/Noun 짜증/Noun 나다/Verb 목소리/Noun 흠/Noun...>


In [9]:
print(len(text.tokens))                 # returns number of tokens
print(len(set(text.tokens)))            # returns number of unique tokens
pprint(text.vocab().most_common(10))    # returns frequency distribution

2194536
48765
[('./Punctuation', 68630),
 ('영화/Noun', 51365),
 ('하다/Verb', 50281),
 ('이/Josa', 39123),
 ('보다/Verb', 34764),
 ('의/Josa', 30480),
 ('../Punctuation', 29055),
 ('에/Josa', 27108),
 ('가/Josa', 26696),
 ('을/Josa', 23481)]


In [12]:
pprint(text.vocab().most_common()[:-20:-1])  #참고, 반대로 잘 안나오는거

[('수간/Noun', 1),
 ('디케이드/Noun', 1),
 ('Carl/Alpha', 1),
 ('http://blog.naver.com/oroblast/220215679580/URL', 1),
 ('찎었/Noun', 1),
 ('십점줌/Noun', 1),
 ('열외/Noun', 1),
 ('축적/Noun', 1),
 ('섹귀들/Noun', 1),
 ('ㅍㄹㄱ/KoreanParticle', 1),
 ('차후/Noun', 1),
 ('차이니즈/Noun', 1),
 ('신해혁명/Noun', 1),
 ('샤오린/Noun', 1),
 ('대병소장/Noun', 1),
 ('초코/Noun', 1),
 ('윤영삼/Noun', 1),
 ('노형욱/Noun', 1),
 ('머싰따능/Noun', 1)]


* Collocations (연어): 인접하게 빈번하게 등장하는 단어 (ex: "text" + "mining")

In [13]:
text.concordance('시간/Noun')  #시간이 쓰인 모든 경우의 문맥을 찾아줌

Displaying 25 of 3899 matches:
rticle 아/Exclamation ../Punctuation 시간/Noun 아깝다/Adjective 재밌다/Adjective 별점/Noun
eanParticle 내/Noun ㅇ/KoreanParticle 시간/Noun 넘다/Verb 아깝다/Adjective 움/Noun ../Pun
tion 평점/Noun 에/Josa 속지/Noun 말다/Verb 시간/Noun 낭비/Noun 돈/Noun 낭비/Noun 임/Noun 리얼리티/
하다/Adjective 난/Noun 별루더/Noun 라/Josa 시간/Noun 낭비/Noun 느낌/Noun 대박/Noun 시청률/Noun 기준
Josa 수작/Noun 이라/Josa 고본/Noun 다/Josa 시간/Noun 이/Josa 아깝다/Adjective 않다/Verb 영화/Nou
Josa 한번/Noun 더/Noun 하다/Verb 되다/Verb 시간/Noun ./Punctuation 그리고/Conjunction 모건/No
/Noun 적/Suffix 계기/Noun 그/Determiner 시간/Noun 이/Josa 표현/Noun 되다/Verb 않다/Verb 시람/N
 좋다/Adjective 뻔하다/Adjective 새벽/Noun 시간/Noun 에/Josa 하다/Verb 일본/Noun 영화/Noun 들/Su
oun 을/Josa 다시/Noun 느끼다/Verb 보다/Verb 시간/Noun 이다/Verb ./Punctuation 장면/Noun 하나/No
실망/Noun ../Punctuation 이/Determiner 시간/Noun 좀/Noun .../Punctuation 밝다/Verb 긍정/N
ctive 드럽/Noun 게/Josa 재미없다/Adjective 시간/Noun 이/Josa 돈/Noun 이다/Josa ?/Punctuation
돈/Noun 이다/Josa ?/Punctuation 내/Noun 시간/Noun 은/Josa 어쩌다/Verb 그저/Adverb 한마디/Noun 
oun 보고/No

## 3. Sentiment classification with term-existance*
여기서는 간단하게 term이 문서에 존재하는지의 유무에 따라 분류를 해보자.

In [14]:
# 여기서는 최빈도 단어 2000개를 피쳐로 사용

selected_words = [f[0] for f in text.vocab().most_common(2000)]
def term_exists(doc):
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

# 시간 단축을 위한 꼼수로 training corpus의 일부만 사용
train_docs = train_docs[:10000]
train_xy = [(term_exists(d), c) for d, c in train_docs]
test_xy = [(term_exists(d), c) for d, c in test_docs]

최빈도 단어들은 의미가 없을 수 있다. 최상위 50개를 제거하고 분석하면?

TF, TF-IDF의 성능은?
코드 최적화(sparse matrix) or scikit-learn의 TfidfVectorizer()를 이용!

다양한 분류기를 이용할 수 있다. (nltk 제공)
- NLTK
  * Naive Bayes Classifiers
  * Decision Tree Classifiers
  * Maximum Entropy Classifiers
  * scikit-learn 등 다른 패키지 분류기도 얼마든지 사용 가능

In [15]:
# Naive Bayes classifier 적용

# 초간단함
classifier = nltk.NaiveBayesClassifier.train(train_xy)
print(nltk.classify.accuracy(classifier, test_xy))

classifier.show_most_informative_features(10)

# 0.80418
# => Most Informative Features
#        exists(수작/Noun) = True                1 : 0      =     38.0 : 1.0
#        exists(최악/Noun) = True                0 : 1      =     30.1 : 1.0
#      exists(♥/Foreign) = True                1 : 0      =     24.5 : 1.0
#        exists(노잼/Noun) = True                0 : 1      =     22.1 : 1.0
#        exists(낭비/Noun) = True                0 : 1      =     19.5 : 1.0
#       exists(쓰레기/Noun) = True                0 : 1      =     19.4 : 1.0
#        exists(여운/Noun) = True                1 : 0      =     18.9 : 1.0
#       exists(발연기/Noun) = True                0 : 1      =     16.9 : 1.0
#         exists(굿/Noun) = True                1 : 0      =     16.9 : 1.0
#       exists(최고다/Noun) = True                1 : 0      =     15.9 : 1.0

0.80416
Most Informative Features
         exists(수작/Noun) = True                1 : 0      =     38.0 : 1.0
         exists(최악/Noun) = True                0 : 1      =     30.1 : 1.0
       exists(♥/Foreign) = True                1 : 0      =     24.5 : 1.0
         exists(노잼/Noun) = True                0 : 1      =     22.1 : 1.0
         exists(낭비/Noun) = True                0 : 1      =     19.5 : 1.0
        exists(쓰레기/Noun) = True                0 : 1      =     19.4 : 1.0
         exists(여운/Noun) = True                1 : 0      =     18.9 : 1.0
        exists(발연기/Noun) = True                0 : 1      =     16.9 : 1.0
          exists(굿/Noun) = True                1 : 0      =     16.9 : 1.0
        exists(최고다/Noun) = True                1 : 0      =     15.9 : 1.0


리뷰 1만개만 쓴 경우에도 accuracy가 0.80이 나왔다. 나름 성공적